In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [16]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307),(0.3081))
])

In [21]:
train_dataset = torchvision.datasets.MNIST(
    "./data", train=True, download=True, transform=transform
)
test_dataset = torchvision.datasets.MNIST(
    "./data", train=False, download=True, transform=transform
)

100%|██████████| 9.91M/9.91M [00:17<00:00, 569kB/s] 
100%|██████████| 28.9k/28.9k [00:00<00:00, 135kB/s]
100%|██████████| 1.65M/1.65M [00:03<00:00, 538kB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 14.7MB/s]


In [22]:
train_loader = DataLoader(train_dataset,batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [23]:
class MNISTClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(784,128),
            nn.ReLU(),
            nn.Linear(128,10)
        )

    def forward(self, x):
        x = self.flatten(x)
        x = self.layers(x)
        return x


In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")


Using cuda


In [26]:
model = MNISTClassifier().to(device=device)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

In [27]:
def train_epoch(model, train_loader,loss_function, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for batch_idx, (data,target) in enumerate(train_loader):
        data,target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output,target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _,predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()
        if batch_idx %  100 == 0 and batch_idx > 0:
            avg_loss = running_loss/100
            acc = 100. * correct/total
            print(f' [{batch_idx * 64}/{60000}]'
                  f' Loss: {avg_loss:.3f} | Accuracy: {acc:.1f} %')
            running_loss = 0.0

In [28]:
def evaluate(model, test_loader,device):
    model.eval()
    correct = 0
    total = 0 
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total = targets.size(0)
            correct = predicted.eq(targets).sum().item()
    
    return 100. * correct/total

In [30]:
num_epochs = 10
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}")
    train_epoch(model,train_loader=train_loader, loss_function=loss_function,optimizer=optimizer,device=device)
    accuracy = evaluate(model=model, test_loader=test_loader, device=device)
    print(f"Test Accuracy: {accuracy:2f}")

Epoch 1
 [6400/60000] Loss: 0.635 | Accuracy: 82.6 %
 [12800/60000] Loss: 0.318 | Accuracy: 86.7 %
 [19200/60000] Loss: 0.265 | Accuracy: 88.5 %
 [25600/60000] Loss: 0.221 | Accuracy: 89.7 %
 [32000/60000] Loss: 0.200 | Accuracy: 90.6 %
 [38400/60000] Loss: 0.194 | Accuracy: 91.2 %
 [44800/60000] Loss: 0.169 | Accuracy: 91.7 %
 [51200/60000] Loss: 0.174 | Accuracy: 92.1 %
 [57600/60000] Loss: 0.152 | Accuracy: 92.5 %
Test Accuracy: 95.700000
Epoch 2
 [6400/60000] Loss: 0.116 | Accuracy: 96.7 %
 [12800/60000] Loss: 0.124 | Accuracy: 96.4 %
 [19200/60000] Loss: 0.115 | Accuracy: 96.4 %
 [25600/60000] Loss: 0.105 | Accuracy: 96.6 %
 [32000/60000] Loss: 0.122 | Accuracy: 96.5 %
 [38400/60000] Loss: 0.118 | Accuracy: 96.5 %
 [44800/60000] Loss: 0.094 | Accuracy: 96.6 %
 [51200/60000] Loss: 0.097 | Accuracy: 96.6 %
 [57600/60000] Loss: 0.099 | Accuracy: 96.7 %
Test Accuracy: 96.800000
Epoch 3
 [6400/60000] Loss: 0.096 | Accuracy: 97.3 %
 [12800/60000] Loss: 0.080 | Accuracy: 97.5 %
 [19200/6